# 사물이미지 분류 경진대회 베이스라인2 

안녕하세요 데이커 여러분! 이전 베이스라인은 잘 보셨나요?

이번 베이스라인에서는 딥러닝 모델 구축 시 많이 사용되는 기법인 전이학습(transfer learning) 에 대해 소개하고자 합니다. 

전이학습이란 이미 구축되어 있는 모델을 사용하는 기법입니다. 

그럼 본격적으로 코드를 통해 전이학습에 대해 알아보아요! 

* 코드를 어떻게 실행시켜야 할지 잘 모르시는 분은 아래 "코랩으로 데이콘 참여하기"를 먼저 봐주세요!
https://dacon.io/competitions/official/235836/talkboard/404882

* 데이터를 살펴보는 탐색적 데이터 분석 (Exploratory Data Analysis, EDA) 코드를 먼저 보고 오시면 좋습니다.

In [2]:
#필요한 library 들을 load 합니다.
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import random
import numpy as np
from tqdm import tqdm

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from multiprocessing import cpu_count
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss
from torchvision.models import efficientnet_b3 as efficientnet
from sklearn.model_selection import train_test_split

# 데이터 load, 전처리

데이터를 load 하기 전에 기본적인 전처리 코드를 작성해줍시다. 

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(), #이미지 데이터를 tensor 데이터 포멧으로 바꾸어줍니다.
    transforms.Resize([224,224]), #이미지의 크기가 다를 수 있으니 크기를 통일해 줍니다.
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) #픽셀 단위 데이터를 정규화 시켜줍니다.
])

데이터를 불러와 줍니다. 

PYTORCH 의 ImageFolder 메소드를 사용하면 folder 의 이름을 자동으로 라벨링이 됩니다.

예를 들어 airplane 이라는 folder 내에 이미지 파일들이 있다면, 이미지 파일들의 라벨을 '0' 으로 라벨링이 됩니다. 

In [4]:
train_data = datasets.ImageFolder(root='./data/train/',transform=transform)

모델 평가를 위해 train 데이터에서 validation 데이터를 나누어줍니다.

In [5]:
train_idx, valid_idx = train_test_split(np.arange(len(train_data)), test_size=0.2, 
                                        random_state=42, shuffle=True, stratify=train_data.targets)

모델에 load 할 batch_size 를 설정해 줍니다. 

batch_size 란 하드웨어에 한번에 load 할 데이터의 크기입니다. 

num_workers 란 데이터 로드 멀티 프로세싱을 위한 파라미터입니다. 

In [6]:
batch_size = 32
num_workers = int(cpu_count() / 2)

data loader 를 생성해줍니다. 

data loader 란 데이터 셋을 순회하며 모델에 데이터를 넣어주는 객체입니다. 

In [7]:
train_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(train_idx), num_workers=num_workers)
valid_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(valid_idx), num_workers=num_workers)

데이터의 크기를 확인해봅니다.

In [8]:
train_total = len(train_idx)
valid_total = len(valid_idx)

train_batches = len(train_loader)
valid_batches = len(valid_loader)

In [9]:
print('total train imgs :',train_total,'/ total train batches :', train_batches)
print('total valid imgs :',valid_total, '/ total valid batches :', valid_batches)

total train imgs : 40000 / total train batches : 1250
total valid imgs : 10001 / total valid batches : 313


# Device 설정

device 를 설정해줍니다.

이번 베이스라인에서는 gpu 가 있다고 가정하겠습니다. 

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.is_available()

True

# 모델 불러오기 / 파라미터 설정

전이학습을 위한 모델을 load 해줍니다. 

PYTORCH 의 models 메소드를 사용하면 손쉽게 외부의 모델을 불러올 수 있습니다.

이번 베이스라인에서는 efficientnet_b3 모델을 사용해 볼 것입니다. 

사전 학습 모델을 사용하는 것은 부정행위에 해당하니, pretrained 파라미터를 False 로 설정해야 합니다!

pretrained 파라미터를 True 로 설정한다면, ImageNet 이라는 데이터셋을 대상으로 학습된 모델이 load 됩니다.

반면, pretrained 파라미터를 False 로 설정한다면, 모델의 구조만 load 되고 모델의 가중치 들은 load 되지 않습니다. 

In [11]:
net = models.efficientnet_b3(pretrained=False)
net.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)

모델에 데이터를 학습하기 위해서는 모델의 마지막 layer 의 output size 와 분류할 라벨의 수를 입력해주어야 합니다.

In [12]:
net.fc = nn.Linear(1000, 10)
net = net.to(device) 

모델의 파라미터들을 설정해줍니다. 

In [ ]:
criterion = CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
epochs = 10

# 학습 

반복문을 이용해 학습을 진행시켜줍니다.

In [13]:
for epoch in range(epochs):
    net.train()
    
    train_loss = 0 
    train_correct = 0
    tqdm_dataset = tqdm(train_loader)
    for x,y in tqdm_dataset:
        x = x.to(device)
        y = y.to(device)
        outputs = net(x)
        loss = criterion(outputs,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()
        
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(loss.item()),
        })

    train_loss = train_loss / train_batches
    train_acc = train_correct / train_total
    
    net.eval()
    
    valid_loss = 0 
    valid_correct = 0
    tqdm_dataset = tqdm(valid_loader)
    with torch.no_grad():
        for x,y in tqdm_dataset:
            x = x.to(device)
            y = y.to(device)
            
            outputs = net(x)
            loss = criterion(outputs, y)
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            valid_correct += predicted.eq(y).sum().item()
            
            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(loss.item()),
            })
            
    valid_loss = valid_loss / valid_batches
    valid_acc = valid_correct / valid_total
        
    print('epochs',epoch+1, 'train loss',train_loss,'train acc', train_acc, 'valid loss',valid_loss, 'valid acc',valid_acc)

100%|██████████| 313/313 [00:10<00:00, 30.08it/s, Epoch=1, Loss=1.362997]


epochs 1 train loss 1.722855820798874 train acc 0.37455 valid loss 1.3101708439592354 valid acc 0.5107489251074893


100%|██████████| 313/313 [00:10<00:00, 29.77it/s, Epoch=2, Loss=0.990702]


epochs 2 train loss 1.1655808423042298 train acc 0.58715 valid loss 0.9792095011415572 valid acc 0.6475352464753524


100%|██████████| 313/313 [00:10<00:00, 29.81it/s, Epoch=3, Loss=0.657171]


...

epochs 8 train loss 0.4534196371734142 train acc 0.84175 valid loss 0.5423957146109103 valid acc 0.822017798220178


100%|██████████| 313/313 [00:10<00:00, 29.64it/s, Epoch=9, Loss=0.505034]


epochs 9 train loss 0.40080086914300916 train acc 0.8616 valid loss 0.5289487347911341 valid acc 0.8208179182081792


100%|██████████| 313/313 [00:10<00:00, 29.65it/s, Epoch=10, Loss=0.613084]


epochs 10 train loss 0.3546241638362408 train acc 0.876975 valid loss 0.5519655163105304 valid acc 0.8181181881811819


# 모델 저장 / 불러오기

학습된 모델의 가중치를 저장합니다. 

In [ ]:
path = './model.pth'
torch.save(net.state_dict(),path)

저장된 모델을 불러옵니다.

In [15]:
path = './model.pth'
net.load_state_dict(torch.load(path))

<All keys matched successfully>

# 추론 

이제 학습이 완료되었습니다! 

그럼 test 데이터를 예측해 보아요.

test 데이터를 불어옵니다.

In [4]:
from glob import glob
import PIL.Image
import numpy as np

test_images = []

path = './data/'
for filename in sorted(glob(path + "test/*.jpg")):
    an_img = PIL.Image.open(filename) 
    img_array = np.array(an_img) 
    test_images.append(img_array) 

test_images = np.array(test_images)

In [17]:
class CustomDataset(Dataset):
    def __init__(self, transform):
        self.transform = transform 
        self.img_list = test_images
        self.img_labels = [0] * 10000 

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        return self.transform(self.img_list[idx]), self.img_labels[idx]

In [18]:
test_set = CustomDataset(transform)

In [19]:
test_loader = DataLoader(test_set, batch_size = batch_size, num_workers=num_workers)

본격적으로 test 데이터를 예측해보아요! 

예측을 할 때는 학습이 진행되지 않도록 net.eval() 코드를 작성해주어야 합니다. 

데이터가 backpropagation 되어 가중치가 수정되지 않도록 해주는 코드입니다. 

In [21]:
import pandas as pd
sample_submission = pd.read_csv('./data/sample_submission.csv')

net.eval()

batch_index = 0

for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    outputs = net(images)
    batch_index = i * batch_size
    max_vals, max_indices = torch.max(outputs, 1)
    sample_submission.iloc[batch_index:batch_index + batch_size, 1:] = max_indices.long().cpu().numpy()[:,np.newaxis]

예측된 데이터의 라벨은 숫자로 되어있습니다. 

train 데이터를 불러올 때 ImageFolder 메소드를 사용해 데이터를 불러왔기 때문입니다. 

제출을 위해 라벨을 다시 복원 시켜 줍니다.

In [23]:
labels = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer',
          5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
sample_submission['target'] = sample_submission['target'].map(labels)

In [25]:
sample_submission.head()

,id,target
0,0000.jpg,horse
1,0001.jpg,airplane
2,0002.jpg,airplane
3,0003.jpg,bird
4,0004.jpg,airplane


In [24]:
sample_submission.to_csv('submit.csv',index=False)

축하합니다! 데이터 분석을 완료하셨습니다!

앞으로도 데이콘과 함께 즐겁게 데이터 분석 능력을 키워가시면 좋겠습니다.

감사합니다.